In [1]:
import spacy 
from spacy.pipeline import EntityRuler
import pandas as pd
import numpy as np 


In [2]:
df = pd.read_csv("../data/sample_resumes/Resume/resumeDataSet2_transformed_filtered.csv")
df = df.reindex(np.random.permutation(df.index))
data = df.copy().iloc[
    0:200,
]
data.head()


,ID,Resume_str,Resume_html,Category
160,604,Core skills â¢ Project / Program Management â...,NaN,DevOps Engineer
312,853,Hobbies â¢ Playing Chess â¢ Solving Rubik's ...,NaN,Blockchain
215,714,Education Details \r\nJanuary 2016 BSc. Mumba...,NaN,Database
95,443,Education Details \r\n\r\nSAP Technical Archit...,NaN,SAP Developer
27,347,Skills â¢ Language: Java â¢ Operating System...,NaN,Java Developer


In [ ]:
# import en_core_web_lg


nlp = spacy.blank("en")
# nlp.add_pipe("ner")

ruler = nlp.add_pipe("entity_ruler")

# ner = nlp.add_pipe("ner")

#Skills for entities 


patterns = [
    {"label": "SOFTWARE", "pattern": "Python"},
    {"label": "SOFTWARE", "pattern": [{"LOWER": "machine"}, {"LOWER": "learning"}]}

]

ruler.add_patterns(patterns)



doc = nlp("Hello I love me some Python")
for ent in doc.ents:
    print(ent.text, ent.label_)

ValueError: [E109] Component 'ner' could not be run. Did you forget to call `initialize()`?